<a href="https://colab.research.google.com/github/C23-PS286-Capstone-Project/skinsight-machine-learning/blob/main/Notebook/SkinSight_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Importing Library For ML**

In [1]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import seaborn as sns
import humanize
import random
import re
import shutil
import math
import multiprocessing
import asyncio
import concurrent.futures

from google.colab import files
from PIL import Image
from tensorflow import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras.models import Sequential, Model
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, GlobalAveragePooling2D
from keras.callbacks import EarlyStopping, Callback
from sklearn.metrics import classification_report

# Mount gdrive

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


## Importing Dataset From Kaggle

Upload kaggle api key

In [ ]:
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"rizkyadityaichwanto","key":"9e8cda76e29af00972c820904e58788e"}'}

In [ ]:
!ls -lha kaggle.json

-rw-r--r-- 1 root root 75 May 19 21:35 kaggle.json


In [ ]:
!pip install -q kaggle

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [ ]:
!chmod 600 /root/.kaggle/kaggle.json

Check dir

In [ ]:
!pwd

/content


Download Datasets & Unzipping

In [ ]:
!kaggle datasets download -d frabbisw/facial-age

 99% 834M/840M [00:13<00:00, 80.0MB/s]
100% 840M/840M [00:13<00:00, 66.6MB/s]


In [ ]:
!unzip facial-age.zip

Streaming output truncated to the last 5000 lines.
  inflating: face_age/face_age/024/7190.png  
  inflating: face_age/face_age/024/7213.png  
  inflating: face_age/face_age/024/7342.png  
  inflating: face_age/face_age/024/7366.png  
  inflating: face_age/face_age/024/7406.png  
  inflating: face_age/face_age/024/7411.png  
  inflating: face_age/face_age/024/7497.png  
  inflating: face_age/face_age/024/7511.png  
  inflating: face_age/face_age/024/7520.png  
  inflating: face_age/face_age/024/7558.png  
  inflating: face_age/face_age/024/7600.png  
  inflating: face_age/face_age/024/7629.png  
  inflating: face_age/face_age/024/765.png  
  inflating: face_age/face_age/024/7691.png  
  inflating: face_age/face_age/024/7771.png  
  inflating: face_age/face_age/024/7782.png  
  inflating: face_age/face_age/024/7817.png  
  inflating: face_age/face_age/024/786.png  
  inflating: face_age/face_age/024/7955.png  
  inflating: face_age/face_age/024/8099.png  
  inflating: face_age/face_age/

# Importing dataset from IMDB-WIKI

### Here the datasets (7GB) : https://data.vision.ee.ethz.ch/cvl/rrothe/imdb-wiki/

In [4]:
!wget https://data.vision.ee.ethz.ch/cvl/rrothe/imdb-wiki/static/imdb_crop.tar

--2023-06-01 23:34:16--  https://data.vision.ee.ethz.ch/cvl/rrothe/imdb-wiki/static/imdb_crop.tar
Resolving data.vision.ee.ethz.ch (data.vision.ee.ethz.ch)... 129.132.52.178, 2001:67c:10ec:36c2::178
Connecting to data.vision.ee.ethz.ch (data.vision.ee.ethz.ch)|129.132.52.178|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7012157440 (6.5G) [application/x-tar]
Saving to: ‘imdb_crop.tar’

imdb_crop.tar       100%[===================>]   6.53G  13.4MB/s    in 7m 59s  

2023-06-01 23:42:17 (13.9 MB/s) - ‘imdb_crop.tar’ saved [7012157440/7012157440]



In [5]:
!tar -xf imdb_crop.tar

In [17]:
!rm -rf agedataset

### Arrange datasets

In [10]:
#Generate annotation for keras
#https://data.vision.ee.ethz.ch/cvl/rrothe/imdb-wiki/

from scipy import io as spio
from datetime import datetime

import numpy as np
import os
import shutil

if(os.path.exists("/content/")):
	DATASET_ROOT_PATH="/content"
else:
	DATASET_ROOT_PATH="./"

OUTPUT_LABEL="agedataset"

IMDB_PATH=DATASET_ROOT_PATH+"/imdb_crop/"
AGE_PATH=DATASET_ROOT_PATH+"/"+OUTPUT_LABEL+"/annotations/age/"
AGE101_PATH=DATASET_ROOT_PATH+"/"+OUTPUT_LABEL+"/annotations/age101/"

if(not os.path.exists(DATASET_ROOT_PATH+"dataset/"+OUTPUT_LABEL+"/")):
	os.mkdir(DATASET_ROOT_PATH+"/"+OUTPUT_LABEL)
if(not os.path.exists(DATASET_ROOT_PATH+"/"+OUTPUT_LABEL+"/annotations/")):
	os.mkdir(DATASET_ROOT_PATH+"/"+OUTPUT_LABEL+"/annotations")

if(not os.path.exists(AGE_PATH)):
	os.mkdir(AGE_PATH)
	os.mkdir(AGE_PATH+"train")
	os.mkdir(AGE_PATH+"train/0-2")
	os.mkdir(AGE_PATH+"train/4-6")
	os.mkdir(AGE_PATH+"train/8-13")
	os.mkdir(AGE_PATH+"train/15-20")
	os.mkdir(AGE_PATH+"train/25-32")
	os.mkdir(AGE_PATH+"train/38-43")
	os.mkdir(AGE_PATH+"train/48-53")
	os.mkdir(AGE_PATH+"train/60-")
	os.mkdir(AGE_PATH+"validation")
	os.mkdir(AGE_PATH+"validation/0-2")
	os.mkdir(AGE_PATH+"validation/4-6")
	os.mkdir(AGE_PATH+"validation/8-13")
	os.mkdir(AGE_PATH+"validation/15-20")
	os.mkdir(AGE_PATH+"validation/25-32")
	os.mkdir(AGE_PATH+"validation/38-43")
	os.mkdir(AGE_PATH+"validation/48-53")
	os.mkdir(AGE_PATH+"validation/60-")

if(not os.path.exists(AGE101_PATH)):
	os.mkdir(AGE101_PATH)
	os.mkdir(AGE101_PATH+"train")
	os.mkdir(AGE101_PATH+"validation")
	for i in range(0,101):
		os.mkdir(AGE101_PATH+"train/"+format(i, '03d'))
		os.mkdir(AGE101_PATH+"validation/"+format(i, '03d'))

def calc_age(taken, dob):
	birth = datetime.fromordinal(max(int(dob) - 366, 1))
	if birth.month < 7:
		return taken - birth.year
	else:
		return taken - birth.year - 1

def is_valid(face_score,second_face_score,age):
	if face_score < 1.0:
		return False
	if (~np.isnan(second_face_score)) and second_face_score > 0.0:
		return False
	if ~(0 <= age <= 100):
		return False	
	return True

def get_age_path(age):
	if(age>=0 and age<=3):
		return "0-2"
	if(age>=4 and age<=7):
		return "4-6"
	if(age>=8 and age<=14):
		return "8-13"
	if(age>=15 and age<=24):
		return "15-20"
	if(age>=25 and age<=37):
		return "25-32"
	if(age>=38 and age<=47):
		return "38-43"
	if(age>=48 and age<=59):
		return "48-53"
	return "60-"

meta = spio.loadmat(IMDB_PATH+"imdb.mat")

db = "imdb"

full_path = meta[db][0, 0]["full_path"][0]
dob = meta[db][0, 0]["dob"][0]
photo_taken = meta[db][0, 0]["photo_taken"][0]
face_score = meta[db][0, 0]["face_score"][0]
second_face_score = meta[db][0, 0]["second_face_score"][0]
age = [calc_age(photo_taken[i], dob[i]) for i in range(len(dob))]

for i in range(len(full_path)):
	if(not is_valid(face_score[i],second_face_score[i],age[i])):
		continue
	print("path:"+str(full_path[i])+" gender:"+str(gender[i]))	
	train_or_validation="train"
	if(i%4==0):
		train_or_validation="validation"
	src_img=IMDB_PATH+full_path[i][0]
	shutil.copyfile(src_img, AGE_PATH+train_or_validation+"/"+get_age_path(age[i])+"/"+str(i)+".jpg")	
	shutil.copyfile(src_img, AGE101_PATH+train_or_validation+"/"+format(age[i],"03d")+"/"+str(i)+".jpg")

Streaming output truncated to the last 5000 lines.
path:['37/nm2395937_rm165318144_1987-4-12_2006.jpg'] gender:0.0
path:['37/nm2395937_rm2019722496_1987-4-12_2011.jpg'] gender:0.0
path:['37/nm2395937_rm2198451968_1987-4-12_2012.jpg'] gender:0.0
path:['37/nm2395937_rm2271004928_1987-4-12_2014.jpg'] gender:0.0
path:['37/nm2395937_rm2282338048_1987-4-12_2012.jpg'] gender:0.0
path:['37/nm2395937_rm2299115264_1987-4-12_2012.jpg'] gender:0.0
path:['37/nm2395937_rm2362963712_1987-4-12_2015.jpg'] gender:0.0
path:['37/nm2395937_rm2426970624_1987-4-12_2011.jpg'] gender:0.0
path:['37/nm2395937_rm2460525056_1987-4-12_2011.jpg'] gender:0.0
path:['37/nm2395937_rm2480404224_1987-4-12_2015.jpg'] gender:0.0
path:['37/nm2395937_rm2661851648_1987-4-12_2011.jpg'] gender:0.0
path:['37/nm2395937_rm275363584_1987-4-12_2012.jpg'] gender:0.0
path:['37/nm2395937_rm3191452416_1987-4-12_2012.jpg'] gender:0.0
path:['37/nm2395937_rm3424317952_1987-4-12_2015.jpg'] gender:0.0
path:['37/nm2395937_rm3450649088_1987-4-1

# Import datasets from UTKFace

### Here (800MB) : https://susanqq.github.io/UTKFace/

In [18]:
!cp ./gdrive/MyDrive/UTKFace.tar.gz /content/

In [19]:
!tar -xf UTKFace.tar.gz

Anotation for dataset

In [20]:
#Generate annotation for keras
#https://susanqq.github.io/UTKFace/

from scipy import io as spio
from datetime import datetime

import numpy as np
import os
import shutil
import glob

if(os.path.exists("/content/")):
	DATASET_ROOT_PATH="/content"
else:
	DATASET_ROOT_PATH="./"

OUTPUT_LABEL="agedataset"

UTKFACE_PATH=DATASET_ROOT_PATH+"/UTKFace/"
AGE_PATH=DATASET_ROOT_PATH+"/"+OUTPUT_LABEL+"/annotations/age/"
AGE101_PATH=DATASET_ROOT_PATH+"/"+OUTPUT_LABEL+"/annotations/age101/"

if(not os.path.exists(DATASET_ROOT_PATH+"/"+OUTPUT_LABEL+"/")):
	os.mkdir(DATASET_ROOT_PATH+"/"+OUTPUT_LABEL)
if(not os.path.exists(DATASET_ROOT_PATH+"/"+OUTPUT_LABEL+"/annotations/")):
	os.mkdir(DATASET_ROOT_PATH+"/"+OUTPUT_LABEL+"/annotations")

if(not os.path.exists(AGE_PATH)):
	os.mkdir(AGE_PATH)
	os.mkdir(AGE_PATH+"train")
	os.mkdir(AGE_PATH+"train/0-2")
	os.mkdir(AGE_PATH+"train/4-6")
	os.mkdir(AGE_PATH+"train/8-13")
	os.mkdir(AGE_PATH+"train/15-20")
	os.mkdir(AGE_PATH+"train/25-32")
	os.mkdir(AGE_PATH+"train/38-43")
	os.mkdir(AGE_PATH+"train/48-53")
	os.mkdir(AGE_PATH+"train/60-")
	os.mkdir(AGE_PATH+"validation")
	os.mkdir(AGE_PATH+"validation/0-2")
	os.mkdir(AGE_PATH+"validation/4-6")
	os.mkdir(AGE_PATH+"validation/8-13")
	os.mkdir(AGE_PATH+"validation/15-20")
	os.mkdir(AGE_PATH+"validation/25-32")
	os.mkdir(AGE_PATH+"validation/38-43")
	os.mkdir(AGE_PATH+"validation/48-53")
	os.mkdir(AGE_PATH+"validation/60-")

if(not os.path.exists(AGE101_PATH)):
	os.mkdir(AGE101_PATH)
	os.mkdir(AGE101_PATH+"train")
	os.mkdir(AGE101_PATH+"validation")
	for i in range(0,101):
		os.mkdir(AGE101_PATH+"train/"+format(i, '03d'))
		os.mkdir(AGE101_PATH+"validation/"+format(i, '03d'))

def get_gender_path(gender):
	if(gender==1):
		return "f"
	return "m"

def get_age_path(age):
	if(age>=0 and age<=3):
		return "0-2"
	if(age>=4 and age<=7):
		return "4-6"
	if(age>=8 and age<=14):
		return "8-13"
	if(age>=15 and age<=24):
		return "15-20"
	if(age>=25 and age<=37):
		return "25-32"
	if(age>=38 and age<=47):
		return "38-43"
	if(age>=48 and age<=59):
		return "48-53"
	return "60-"

i=0
for image_path in glob.glob(UTKFACE_PATH+"*.jpg"):
	image_name = os.path.basename(image_path)
	print(image_name)
	age, gender = image_name.split("_")[:2]
	gender=int(gender)
	age=min(int(age), 100)

	train_or_validation="train"
	if(i%4==0):
		train_or_validation="validation"

	src_img=UTKFACE_PATH+image_name
	shutil.copyfile(src_img, AGE_PATH+train_or_validation+"/"+get_age_path(age)+"/"+str(i)+".jpg")	
	shutil.copyfile(src_img, AGE101_PATH+train_or_validation+"/"+format(age,"03d")+"/"+str(i)+".jpg")

	i=i+1

Streaming output truncated to the last 5000 lines.
25_0_1_20170113132636784.jpg.chip.jpg
27_0_0_20170117010746791.jpg.chip.jpg
26_1_2_20170116180346479.jpg.chip.jpg
54_1_3_20170117191109210.jpg.chip.jpg
24_0_3_20170119165927582.jpg.chip.jpg
6_1_4_20161221195807162.jpg.chip.jpg
22_0_2_20170116172604316.jpg.chip.jpg
21_1_2_20170116171009058.jpg.chip.jpg
23_1_0_20170116222839270.jpg.chip.jpg
30_1_1_20170113011807912.jpg.chip.jpg
36_1_0_20170110173815028.jpg.chip.jpg
24_1_2_20170116222629764.jpg.chip.jpg
76_1_3_20170110181654947.jpg.chip.jpg
47_1_1_20170113002000875.jpg.chip.jpg
80_0_0_20170117204632601.jpg.chip.jpg
23_1_2_20170104015902116.jpg.chip.jpg
58_0_0_20170120224926808.jpg.chip.jpg
53_1_0_20170120223149451.jpg.chip.jpg
26_0_0_20170103181224784.jpg.chip.jpg
38_0_0_20170117184513097.jpg.chip.jpg
8_1_2_20170109203443934.jpg.chip.jpg
73_1_0_20170120230335947.jpg.chip.jpg
1_0_3_20161220222642427.jpg.chip.jpg
28_0_1_20170113195927894.jpg.chip.jpg
63_0_0_20170120223104851.jpg.chip.jpg
35

# Let's Start Here

In [21]:
# Define train & val dir
base_dir = './agedataset'
train_dir = base_dir + '/annotations/age/train'
val_dir = base_dir + '/annotations/age/validation'


def train_val_generator(TRAINING_DIR, VALIDATION_DIR):
  # Create imagegen
  train_datagen = ImageDataGenerator(rescale = 1./255.,
                                      rotation_range = 40,
                                      width_shift_range = 0.2,
                                      height_shift_range = 0.2,
                                      shear_range = 0.2,
                                      zoom_range = 0.2,
                                      horizontal_flip = True)

  train_generator = train_datagen.flow_from_directory(directory=TRAINING_DIR,
                                                        batch_size=32, 
                                                        class_mode='categorical',
                                                        target_size=(224, 224))

  test_datagen = ImageDataGenerator(rescale = 1./255.)

  validation_generator = test_datagen.flow_from_directory(directory=VALIDATION_DIR,
                                                        batch_size=32, 
                                                        class_mode='categorical',
                                                        target_size=(224, 224))
  return train_generator, validation_generator

train_generator, validation_generator = train_val_generator(train_dir, val_dir)

Found 17781 images belonging to 8 classes.
Found 5927 images belonging to 8 classes.


In [ ]:
print(train_generator, validation_generator)

<keras.preprocessing.image.DirectoryIterator object at 0x7f492282fe80> <keras.preprocessing.image.DirectoryIterator object at 0x7f4922142d10>


## Transfer Learning using  VGG16

In [7]:
from keras.applications import VGG16

vgg = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the layers of the VGG16 model
for layer in vgg.layers:
    layer.trainable = False

58889256/58889256 [==============================] - 0s 0us/step


### Implement VGG Model in CNN Model

In [8]:
model = Sequential([      
      # VGG Layer
      vgg,
      Flatten(),
      Dropout(0.5),
      # 512 neuron hidden layer      
      Dense(1024, activation='relu'),      
      Dense(512, activation='relu'),
      Dense(8, activation='softmax')
  ])


### Using MobileNetV2

In [ ]:
from keras.applications import MobileNetV2

mobileNetV2 = MobileNetV2(
    input_shape=(224, 224, 3),  
    include_top=False,
    weights="imagenet",    
)

# Freeze the layers of the MobileNetV2 model
mobileNetV2.trainable = False

9406464/9406464 [==============================] - 1s 0us/step


In [ ]:
model = Sequential([      
      # MobileNetV2      
      mobileNetV2,
      # Average Pooling 2D
      GlobalAveragePooling2D(),                        
      Dense(8, activation='softmax')
  ])

In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 7, 7, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 25088)             0         
                                                                 
 dropout (Dropout)           (None, 25088)             0         
                                                                 
 dense (Dense)               (None, 1024)              25691136  
                                                                 
 dense_1 (Dense)             (None, 512)               524800    
                                                                 
 dense_2 (Dense)             (None, 8)                 4104      
                                                                 
Total params: 40,934,728
Trainable params: 26,220,040
No

In [ ]:
model = Sequential([      
      # This is the first convolution
      tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(150, 150, 3)),
      tf.keras.layers.MaxPooling2D(2, 2),
      # The second convolution
      tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
      tf.keras.layers.MaxPooling2D(2,2),
      # The third convolution
      tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
      tf.keras.layers.MaxPooling2D(2,2),
      # The fourth convolution
      tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
      tf.keras.layers.MaxPooling2D(2,2),
      # Flatten the results to feed into a DNN
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dropout(0.5),
      # 512 neuron hidden layer
      tf.keras.layers.Dense(512, activation='relu'),
      tf.keras.layers.Dense(99, activation='softmax')
  ])


In [15]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 7, 7, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 25088)             0         
                                                                 
 dropout (Dropout)           (None, 25088)             0         
                                                                 
 dense (Dense)               (None, 1024)              25691136  
                                                                 
 dense_1 (Dense)             (None, 512)               524800    
                                                                 
 dense_2 (Dense)             (None, 8)                 4104      
                                                                 
Total params: 40,934,728
Trainable params: 26,220,040
No

## Set Callback

In [12]:
# Set callback if accuracy above 99%
class AccuracyCallback(Callback):
    def on_epoch_end(self, epoch, logs=None):
        if logs['accuracy'] >= 0.99:
            print("\nReached 99% accuracy. Stopping training.")
            self.model.stop_training = True

# Create an instance of the callback
accuracy_callback = AccuracyCallback()

In [22]:
# Define the checkpoint directory and file path
checkpoint_dir = 'gdrive/MyDrive/checkpoints'
checkpoint_path = checkpoint_dir + '/aging_model_checkpoint.h5'

# Create a ModelCheckpoint callback to save the model weights
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    checkpoint_path, monitor='val_loss', save_best_only=True, mode='min', verbose=1
)

# Create an EarlyStopping callback to stop training if validation loss doesn't improve
early_stopping_callback = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', patience=5, mode='min', verbose=1
)

# Create a list of callbacks to pass to the `fit` function
callbacks = [checkpoint_callback, accuracy_callback]

In [23]:
model = tf.keras.models.load_model('./gdrive/MyDrive/checkpoints/aging_model_checkpoint.h5')

In [ ]:
model.compile(loss = 'categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

epochs = 100

model.fit(train_generator,
          epochs=epochs,
          callbacks=callbacks,
          validation_data=validation_generator)

Epoch 1/100
556/556 [==============================] - ETA: 0s - loss: 1.2851 - accuracy: 0.5214
Epoch 1: val_loss improved from inf to 1.17519, saving model to gdrive/MyDrive/checkpoints/aging_model_checkpoint.h5
556/556 [==============================] - 270s 483ms/step - loss: 1.2851 - accuracy: 0.5214 - val_loss: 1.1752 - val_accuracy: 0.5463
Epoch 2/100
556/556 [==============================] - ETA: 0s - loss: 1.2744 - accuracy: 0.5209
Epoch 2: val_loss did not improve from 1.17519
556/556 [==============================] - 255s 458ms/step - loss: 1.2744 - accuracy: 0.5209 - val_loss: 1.2117 - val_accuracy: 0.5477
Epoch 3/100
556/556 [==============================] - ETA: 0s - loss: 1.2758 - accuracy: 0.5187
Epoch 3: val_loss did not improve from 1.17519
556/556 [==============================] - 253s 455ms/step - loss: 1.2758 - accuracy: 0.5187 - val_loss: 1.1863 - val_accuracy: 0.5325
Epoch 4/100
556/556 [==============================] - ETA: 0s - loss: 1.2768 - accuracy: 0.5

In [ ]:
model.save('aging_classification_model.h5')

Load Model & Convert to TFlite

In [ ]:
model = tf.keras.models.load_model('/content/animall_person_other_v2_fine_tuned.h5')
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
open("animall_person_other_v2_fine_tuned.tflite", "wb").write(tflite_model)